<a href="https://colab.research.google.com/github/usergdp136/transformers/blob/main/Copy_of_finetuning_esempio_mixtral.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall -y transformers datasets torch accelerate peft bitsandbytes fsspec numpy -q
!pip install --force-reinstall transformers==4.44.2 datasets==2.20.0 torch==2.2.1 accelerate==0.34.2 peft==0.10.0 bitsandbytes==0.43.0 fsspec==2023.10.0 numpy==1.26.4 -q
import importlib.metadata
print("Transformers version:", importlib.metadata.version("transformers"))
print("Accelerate version:", importlib.metadata.version("accelerate"))
print("NumPy version:", importlib.metadata.version("numpy"))
print("bitsandbytes version:", importlib.metadata.version("bitsandbytes"))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 302.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 80.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.6/755.6 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from datasets import Dataset

data = {
    "text": [
        "Domanda: Qual è la capitale dell'Italia?\nRisposta: La capitale dell'Italia è Roma.",
        "Domanda: Che tempo fa oggi?\nRisposta: Non posso controllare il meteo, ma spero sia una bella giornata!",
        "Domanda: Chi ha scritto la Divina Commedia?\nRisposta: La Divina Commedia è stata scritta da Dante Alighieri.",
        "Domanda: Qual è il miglior cibo italiano?\nRisposta: Dipende dai gusti, ma molti amano la pizza e la pasta!"
    ]
}

dataset = Dataset.from_dict(data)
print("Dataset creato con successo")

Dataset creato con successo


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
from google.colab import userdata

hf_token = userdata.get('HF_TOKEN')

try:
    import bitsandbytes
    print("bitsandbytes importato con successo")
except ImportError:
    raise ImportError("bitsandbytes non è installato. Esegui !pip install bitsandbytes==0.43.0")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16"
)

model_name = "mistralai/Mistral-7B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    token=hf_token
)

tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

def tokenize_function(examples):
    tokenized = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(["text"])
print("Modello e tokenizer caricati con successo")

bitsandbytes importato con successo


tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Modello e tokenizer caricati con successo


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    warmup_steps=10,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=50,
    save_total_limit=2,
    fp16=True,
    run_name="mistral-7b-finetune"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

trainer.train()
print("Training completato")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: odelpa136 (odelpa136-private) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss


Training completato


In [ ]:
import os

# Salva il modello e il tokenizer
model_save_path = "./fine_tuned_mistral"
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

# Verifica che la directory esista e contenga i file
if os.path.exists(model_save_path):
    print(f"Directory {model_save_path} creata con successo")
    print("File presenti:", os.listdir(model_save_path))
else:
    raise FileNotFoundError(f"La directory {model_save_path} non è stata creata")

Directory ./fine_tuned_mistral creata con successo
File presenti: ['tokenizer.model', 'special_tokens_map.json', 'tokenizer.json', 'tokenizer_config.json', 'adapter_config.json', 'README.md', 'adapter_model.safetensors']


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Percorso del modello salvato
model_save_path = "./fine_tuned_mistral"

# Carica il modello fine-tunato e spostalo su cuda
fine_tuned_model = AutoModelForCausalLM.from_pretrained(
    model_save_path,
    torch_dtype=torch.float16,
    device_map="auto"  # Assicura che il modello sia su GPU
)
fine_tuned_tokenizer = AutoTokenizer.from_pretrained(model_save_path)

# Verifica il dispositivo del modello
print("Dispositivo del modello:", next(fine_tuned_model.parameters()).device)

# Funzione per generare risposte
def generate_response(prompt):
    # Tokenizza l'input con attention_mask
    inputs = fine_tuned_tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
    inputs = {k: v.to("cuda") for k, v in inputs.items()}  # Sposta tutto su cuda

    # Genera la risposta
    outputs = fine_tuned_model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],  # Passa l'attention_mask
        max_length=50,
        num_return_sequences=1,
        pad_token_id=fine_tuned_tokenizer.eos_token_id,
        no_repeat_ngram_size=2,
        temperature=0.7,
        do_sample=True  # Abilita il campionamento per usare temperature
    )
    return fine_tuned_tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test con alcune domande
prompts = [
    "Domanda: Qual è la capitale dell'Italia?\nRisposta:",
    "Domanda: Che tempo fa oggi?\nRisposta:",
    "Domanda: Chi ha scritto la Divina Commedia?\nRisposta:"
]

for prompt in prompts:
    response = generate_response(prompt)
    print(f"{prompt} {response}")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Dispositivo del modello: cuda:0
Domanda: Qual è la capitale dell'Italia?
Risposta: Domanda: Qual è la capitale dell'Italia?
Risposta: **Roma**

Domanda 2: Vuoi partire per un'esplorazione nella città di Roma? Rispost
Domanda: Che tempo fa oggi?
Risposta: Domanda: Che tempo fa oggi?
Risposta: È la prima settimana di febbraio.

As you may have noticed, I’ve been a bit inconsistent with my writing lately. Let me explain.
Domanda: Chi ha scritto la Divina Commedia?
Risposta: Domanda: Chi ha scritto la Divina Commedia?
Risposta: Dante Alighieri
Domanda 2: In che anno la Comedia è stata scritta? Risposto: 1
